In [ ]:
%pylab inline

In [ ]:
import katpoint
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator, MultipleLocator
import matplotlib.dates as mdates
from matplotlib.font_manager import FontProperties
from astropy.time import Time
import os
from IPython.core.display import display, HTML

In [ ]:
#create a variety of coloured markers.  After a while the plots get really crowded,
#but you can add on to this if you like.
markers = []
colors = ['b','g','r','c','m','y','k']
pointtypes = ['o','*','x','^','s','p','h','+','D','d','v','H','d','v']
for point in  pointtypes:
    for color in colors:
        markers.append(str(color+point))

This notebook can be used each week to plan the scheduling of observations on MeerKAT. 

It will generate an obs table, with times in SAST, that can be used to block off times in the Google calendar

See https://en.wikipedia.org/wiki/Sidereal_time for a discussion of siderial time

Some useful points to check your scheduling sanity:
* The RA of a target is equal to the LST when the target is at its highest elevation (when it tranits)... this is useful because if you are doing scheduling the best is to observe equally on either side of this time
* LST will change by roughy 4 mins per day ... so a target will transit 4 mins earlier each day
* MeerKAT LST matches SAST on roughly 30 Sep each year ... it is 6 hrs after SAST on about 30 Dec, 12 hrs difference in March and 6 hrs before in June 

You can also look at the following pyephem doc page for more ways to use the ant.ref_observer object. http://rhodesmill.org/pyephem/quick.html

Link to the MeerKAT observation calendar

https://calendar.google.com/calendar/embed?src=ska.ac.za_hd1ilp7tkoffoa9f2gh0j6l0q8%40group.calendar.google.com&ctz=Africa/Johannesburg

In [ ]:
ant = katpoint.Antenna('Ref, -30:42:39.8, 21:26:38.0, 1035.0') # the MeerKAT reference point
ant.ref_observer.horizon = '20:00:00' # horizon set to 20 degrees
SAST = matplotlib.dateutil.relativedelta.relativedelta(hours=2) # useful to convert to SAST (UTC + 2)

In [ ]:
# catalogue cell

cat = katpoint.Catalogue()
cat.antenna = ant
cat.add(file('/home/tony/Desktop/PythonScripts/KAT/cals_Lband.csv'))

In [ ]:
# start time cell

# select the timestamp to be used to generate the obs table 
# comment out one of the following two lines
start_timestamp = katpoint.Timestamp() # use NOW as the start time for the obs table
#start_timestamp = katpoint.Timestamp('2017-10-13 08:00:00') # use a specific time as the start time (in UTC ... )

In [ ]:
# pyephem version of the katpoint timestamp for the ref_observer object
start_ed = start_timestamp.to_ephem_date()

In [ ]:
# generate an html table for displaying
# only the first 16 characters [:16] are shown for the time string
# for readability

h = '<table>'
h += "<tr><td colspan='4'>Obs table for %s</td></tr>"%str(start_timestamp+7200)[:16]
h += "<tr><td>Target<td>Next Rise<td>Next Transit<td>Next Set</tr>"
for tar in cat.targets:
    try:
        rise_time = str((ant.ref_observer.next_rising(tar.body,start_ed).datetime()+SAST))[:16]
    except:
        rise_time = 'source does not rise'
        
    try:
        transit_time = str((ant.ref_observer.next_transit(tar.body,start_ed).datetime()+SAST))[:16]
    except:
        transit_time = 'source does not transit'
    try:
        set_time = str((ant.ref_observer.next_setting(tar.body,start_ed).datetime()+SAST))[:16]
    except:
        set_time = 'source does not set'
    h += '<tr><td> %s | %s <td> %s <td> %s <td> %s </tr>'%(tar.name,tar.aliases,
                                rise_time,
                                transit_time,
                                set_time)
h += "<tr><td colspan='4'>These times are in SAST</td></tr>"
h += "</table>"
display(HTML(h))